<span style="font-size: 24px;"> House Prices Regression </span> 

We will be predicting house prices using XG Boost.

Key skills covered: one hot encoding, normalisation methods, XGboost algorithm, Principle Component Analysis, 
    
<span style="font-size: 20px;"> Dataset </span> 

There are 79 input features, 1460 data points.

The following variables will be dropped due to a high prevalence of N/A values:
- Alley
- MiscFeature
- Fence
- PoolOC
- PoolArea
- MiscVal
- GarageYrBlt

Other preprocessing tasks:

- LotFrontage: Linear feet of street connected to property. Lots of N/A values. I'll set them to 0 if N/A.
- OpenPorchSF: Open porch area in square feet. CREATE A NEW COL CALLED SUM_PORCH_AREA AND COMBINE THIS COL WITH EnclosedPorch, 3SsnPorch and ScreenPorch

<span style="font-size: 20px;"> Strategy </span> 

I'm going to use PCA for dimensionality reduction, it isn't strictly required as we are using an ensemble method called XG Boost which can cope with a high number of features.

Before I do that I'll need to normalise the data.

<b> Data Normalisation </b>

There are three typical methods for Data Normalisation:
- Min-Max scaling scales features to a specific range, usually [0, 1]. X_normalized = (X - X_min) / (X_max - X_min)
- Standardisation is just the usual normalisation. It's good for when the data isn't currently a normal distribution. X_standardized = (X - X_mean) / X_std
- Robust scaling scales features using the median and interquartile range (IQR) instead of the mean and standard deviation. X_robust = (X - X_median) / IQR. This is good for when there are definitely outliers. Why? Because outliers affect mean and standard deviation.

In [179]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

path_train = "/home/frances/Documents/ML preparation/House_Prices_Regression/train.csv"
path_test = "/home/frances/Documents/ML preparation/House_Prices_Regression/test.csv"

df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

categorical_vars = [
    "MSSubClass", "MSZoning", "Street", "LotShape", "LandContour", "Utilities", "LotConfig",
    "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle",
    "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "ExterQual", "ExterCond", "Foundation",
    "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC",
    "CentralAir", "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageFinish",
    "GarageQual", "GarageCond", "PavedDrive", "SaleType", "SaleCondition", 
]

numerical_vars = [
    "LotFrontage", "LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd",
    "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF",
    "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr",
    "TotRmsAbvGrd", "Fireplaces", "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF",
    "EnclosedPorch", "3SsnPorch", "ScreenPorch", "MoSold", "YrSold"
]

# we split the data into categorical and numerical variables

train_categorical = df_train[categorical_vars]
train_numerical = df_train[numerical_vars]
test_categorical = df_test[categorical_vars]
test_numerical = df_test[numerical_vars]

# we encode the variables 
train_encoded = pd.get_dummies(train_categorical, columns=categorical_vars)
test_encoded = pd.get_dummies(test_categorical, columns=categorical_vars)

# we initialise the scaler to normalise the numerical data
scaler = StandardScaler()

test_standardised = scaler.fit_transform(test_numerical) # this returns a numpy array. We don't technically have to convert it back as the model will accept numpy arrays, but for consistency we will.
test_standardised = pd.DataFrame(test_standardised, columns=numerical_vars)
train_standardised = scaler.fit_transform(train_numerical) # this returns a numpy array. We don't technically have to convert it back as the model will accept numpy arrays, but for consistency we will.
train_standardised = pd.DataFrame(train_standardised, columns=numerical_vars)

# we join the numerical and categorical data up once more
X_train = pd.concat([train_standardised, train_encoded],axis=1)
X_test = pd.concat([test_standardised, test_encoded],axis=1)

X_train['LotFrontage'] = X_train['LotFrontage'].fillna(0)
X_train['SUM_PORCH_AREA'] = X_train['OpenPorchSF'] + X_train['EnclosedPorch'] + X_train['3SsnPorch'] + X_train['ScreenPorch']
X_test['LotFrontage'] = X_test['LotFrontage'].fillna(0)
X_test['SUM_PORCH_AREA'] = X_test['OpenPorchSF'] + X_test['EnclosedPorch'] + X_test['3SsnPorch'] + X_test['ScreenPorch']

X_train.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'],axis=1)
X_test.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'],axis=1)

temp = pd.concat([X_train, df_train['SalePrice']],axis=1)
X_train = temp.dropna()
X_test = X_test.dropna()
X_train = temp.drop(['SalePrice'],axis=1)
Y_train = temp['SalePrice']

<span style="font-size: 20px;"> Theory behind Principle Component Analysis </span> 

- The data first needs to be processed and standardised.
- The covariance matrix is calculated. The covariance between two features i and j with n data points is computed as below.

Computation: Take row 1, take feature i in that row, minus the average value for the column. Do the same for the other feature, then multiply them. Do that for all rows, adding them up, then divide by n-1. The fact that it's n-1 and not n is known as Bessel's correction. It makes the covariance an unbiased estimator (statistics stuff).

This tells us how much one variable varies with another. Had we not normalised the variables, it wouldn't be particularly useful

- Then we compute eigenvalues of the covariance matrix, sorting and selecting the k largest.

Eigenvectors represent directions in the original feature space. These eigenvectors (principle components) point in the directions of maximum variance (spread) in the feature space. Note, a feature vector would be a vector where each entry is a numerical value corresponding to a given feature. The feature space is like a set of all possible combinations of those values. If you tried to flatten a dimension where there's high variance, you'd be removing a lot of datapoints. but if you flatten a dimension with low variance, you don't remove much data. This means that PC's help us find the most important features, and allow us to reduce dimensions with too much loss of data.

<b> Methods to choose k </b>

There are a few different methods you can use to determine the value of k. We'll focus on one in particular, the Explained Variance Ratio. Suppose we take the cumulatively keep adding eigenvalues together, dividing the total sum of eigenvectors, and plot this as we add more and more. This is known as the explained variance ratio. We'd like to get to the point where we capture 95% or even 99% of the variance.

- Then we project the original data onto the subspace defined by the select PCs. 

Note that there is another dimensionality reduction method which is typically used for classification models, called Linear Discriminant Analysis (LDA).

In [180]:
import numpy as np
from sklearn.decomposition import PCA

# initialise PCA
pca = PCA()

# fit the pca to the training data
pca.fit(X_train)

# Calculate the explained variance ratio and cumulative explained variance ratio. This uses the fitted model.
exp = pca.explained_variance_ratio_
cum = np.cumsum(exp) # this is an array of floats 

# Note that cum >= 0.95 creates a boolean array where each cumulative explained variance is checked to see if it's higher than 0.95
# We then use argmax to find the index, which gives us k

num_components = np.argmax(cum >= 0.95) + 1

print(num_components)
print(np.shape(X_train))

74
(1460, 286)


It turns out that out of 286 features, 74 of them are sufficient to explain 95% of the variance.

<span style="font-size: 20px;"> Theory behind Gradient Boosting Regression </span> 

1) The model initialises the prediction vector y_pred e.g. as the average value found in the training data for regression, or the natural logarithm of the odds ratio (Probability of Event/Probability of Non-Event) for classification.

2) The model calculates the residuals (errors) and uses these as new y train values, with which to fit a decision tree. 

What this does is it generates predicted residual (error) values, which we then add to y_pred, so we have y_pred + res_pred. Now we repeat this iteratively, with weak decision trees (3/4/5 layers). These are called 'weak learners'. The number of times we iterate (the number of weak learners) is a hyperparameter.

The decision tree predicts these values on the basis of minimising the objective function, which is the negative log loss for classification and the mean squared error for regression.

As with Random Forests, XGBoost does incorporate randomness in feature selection and data subsampling, but this randomness is not a central aspect of the boosting process itself. Instead, it's introduced as a regularization technique to prevent overfitting and improve the diversity of the ensemble.

In [181]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Fit PCA on training data
pca_reduced = PCA(74) # initialises a PCA which knows to keep 74 components

pca_reduced.fit(X_train) # calculates principle components

X_train_pca = pca_reduced.fit_transform(X_train) # this transforms the training set using the principle components

# Split data into training and evaluation sets
#X_train_pca, X_eval_pca, Y_train, Y_eval = train_test_split(X_train_pca, Y_train, test_size=0.20, random_state=42)

# Initialize XGBoost regressor
model = xgb.XGBRegressor()

# Train the model
model.fit(X_train, Y_train)

y_train_pred = model.predict(X_train)

# Evaluate the model
score = model.score(X_train, Y_train) # Note that this is the R^2. It represents the variance that the model captures. It's a value between 0 and 1. Higher values indiciate a better fit.
print("Model Score:", score)

# Assuming you have actual target values (Y_actual) and predicted values (Y_pred)
n = len(Y_train)
mse = np.sum((Y_train - y_train_pred)**2) / n

print("mse:", mse)

Model Score: 0.9995733051136423
mse: 2691074.4387079733


In [168]:
# A score of 99% for an R^2 value indicates a lot of overfitting. Let's see if we can tune the hyperparameters to make it better.

from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': np.arange(100, 1001, 100),# no. of weak learners/iterations
    'max_depth': np.arange(3, 10), # how deep the trees can go
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], # a scaling factor which scales each weak learners input
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], # fraction of sample used for a tree
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], # fraction of features
    'gamma': [0, 0.1, 0.2, 0.3, 0.4], # regularisation parameter, 
}

# Initialize XGBoost regressor
model = xgb.XGBRegressor()

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=100,
    scoring='neg_mean_squared_error',  # Use appropriate metric
    cv=3,  # Cross-validation folds
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Perform random search
random_search.fit(X_train_pca, Y_train)

# Print the best parameters and corresponding score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best Parameters: {'subsample': 0.6, 'n_estimators': 900, 'max_depth': 8, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.9}
Best Score: -795783707.0022014


<span style="font-size: 20px;"> Next steps </span> 

So with or without dimensionality reduction, there's still high overfitting and large error. Not sure what happened there.